<a href="https://colab.research.google.com/github/anujgupta82/NLP_Bootcamp/blob/V_2_0/module2/SkipGram/SkipGram_barebones_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.11-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.11-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.11-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
!pip install mpld3==0.3
!pip install --user "git+https://github.com/javadba/mpld3@display_fix"

     |████████████████████████████████| 798kB 2.8MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=9f6f7f263967f8c8551f8f5bbd8c35df86c178d79b9aac7cbd139c4b58835a66
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
Successfully built mpld3
  Cloning https://github.com/javadba/mpld3 (to revision display_fix) to /tmp/pip-req-build-621xkdpv
  Running command git clone -q https://github.com/javadba/mpld3 /tmp/pip-req-build-621xkdpv
  Running command git checkout -b display_fix --track origin/display_fix
  Switched to a new branch 'display_fix'
  Branch 'display_fix' set up to track remote branch 'display_fix' from 'origin'.
  Running command git submodule update --init --recursive -q
  Created wheel for mpld3: filename=mpld3-0.3.1.dev1-cp36-none-any.whl size=116957 sha256=998fd1ef0d0a423f97fd5261f1d5430bd512eccf5851f53a435ecf171a563ed1
  Stored in directory: /tmp/pip-ephem-wheel-cache-99rqh1vd/wh

In [0]:
import mpld3
mpld3.enable_notebook()

In [0]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

In [0]:
import sys
import numpy as np 
import random
import math
import tensorflow as tf
import matplotlib.pyplot as plt 

sys.path.append("/content/drive/GoogleDrive_Utils/")

In [0]:
from readWikiData import get_wikipedia_data

#### Get data 

In [0]:
sentences, word2idx, idx2word, _ = get_wikipedia_data(n_files=10, n_vocab=1000, by_paragraph=True)

In [0]:
def get_wiki_data_skip_gram(sentences, word2idx, window_size=5):
    training_data = []
    vocab_size = len(word2idx)
    for sentence in sentences:
        if len(sentence) < window_size * 2 + 1:
            continue
        for i in range(len(sentence)):
            left_context = sentence[max(i-window_size, 0): i]
            right_context = sentence[i+1:window_size + i + 1]
            centre = sentence[i]
            
            if len(left_context + right_context) < (2*window_size):
                len_left = len(left_context)
                len_right = len(right_context)
                
                if len_left < len_right:
                    right_context = sentence[i+1 : window_size + i + 1 + (len_right - len_left)]
                else:
                    left_context = sentence[max(i-window_size - (len_left - len_right), 0): i]
            
            temp = left_context + right_context
            
            if len(temp) < window_size * 2:
                print (sentence)
                print (left_context)
                print (right_context)
                print (centre)
                break 
            
            training_data.append((centre, tuple(temp)))
            
            
    print (training_data[:10])
    training_data = list(set(training_data))
    idx2word = {v:k for k, v in word2idx.items()}
    return len(word2idx), training_data, word2idx, idx2word

In [11]:
vocab_size, training_data, word2idx, idx2word = get_wiki_data_skip_gram(sentences, word2idx)

[(1000, (8, 7, 227, 949, 12, 1000, 1000, 1000, 161, 15)), (8, (1000, 7, 227, 949, 12, 1000, 1000, 1000, 161, 15)), (7, (1000, 8, 227, 949, 12, 1000, 1000, 1000, 161, 15)), (227, (1000, 8, 7, 949, 12, 1000, 1000, 1000, 161, 15)), (949, (1000, 8, 7, 227, 12, 1000, 1000, 1000, 161, 15)), (12, (1000, 8, 7, 227, 949, 1000, 1000, 1000, 161, 15)), (1000, (8, 7, 227, 949, 12, 1000, 1000, 161, 15, 1000)), (1000, (7, 227, 949, 12, 1000, 1000, 161, 15, 1000, 1000)), (1000, (227, 949, 12, 1000, 1000, 161, 15, 1000, 1000, 55)), (161, (949, 12, 1000, 1000, 1000, 15, 1000, 1000, 55, 16))]


In [12]:
len(training_data)

11880092

In [13]:
training_data[:10]

[(6, (1000, 1000, 7, 8, 1000, 2, 1000, 1000, 3, 48)),
 (98, (1000, 2, 170, 50, 1000, 12, 1000, 63, 28, 1000)),
 (2, (2, 1000, 1000, 1000, 3, 1000, 1000, 5, 153, 201)),
 (14, (431, 1000, 13, 1000, 1000, 1000, 1000, 55, 653, 27)),
 (313, (2, 1000, 1000, 2, 1000, 383, 19, 52, 2, 1000)),
 (1000, (1000, 1000, 1000, 1000, 1000, 205, 20, 246, 6, 21)),
 (1000, (1000, 1000, 458, 1000, 9, 3, 2, 1000, 1000, 1000)),
 (1000, (1000, 54, 16, 59, 2, 1000, 3, 282, 14, 1000)),
 (4, (6, 1000, 708, 1000, 36, 7, 526, 180, 1000, 12)),
 (310, (382, 1000, 2, 1000, 3, 7, 155, 8, 429, 11))]

##### Get batches

In [0]:
bucket_list = []

def getNextBatchSkipGram(bi_grams_, window_size=5, batch_size=10000):
    global bucket_list
    docs_ids_to_select = list(set(bi_grams_) - set(bucket_list))
    
    if len(docs_ids_to_select) < batch_size:
        bucket_list = []
        docs_ids_to_select = bi_grams_
        
    # Initialize two variables 
    train_X = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    train_label = np.ndarray(shape=(batch_size, window_size*2), dtype=np.int32)
    
    # Get a random set of docs 
    random_docs = random.sample(docs_ids_to_select, batch_size)
    bucket_list += random_docs
    
    
    index = 0 
    
    # Iterate threw all the docs 
    for item in random_docs:
        train_X[index] = item[0]
        train_label[index] = item[1]  
        index += 1
            
    return train_X, train_label

In [15]:
getNextBatchSkipGram(training_data)

(array([[   2],
        [1000],
        [1000],
        ...,
        [1000],
        [   2],
        [1000]], dtype=int32),
 array([[1000,   64,    2, ...,    5, 1000,   14],
        [1000, 1000, 1000, ..., 1000,   10, 1000],
        [   7,  764,    6, ..., 1000, 1000, 1000],
        ...,
        [1000,   10, 1000, ..., 1000, 1000, 1000],
        [ 444,   84,    2, ...,    2,   48,  354],
        [  16,   45,   13, ..., 1000, 1000,   16]], dtype=int32))

##### Let's design the graph for skip gram model 

In [0]:
def init_weight(Mi, Mo):
    shape_sum = float(Mi + Mo) 
    return np.random.uniform(-np.sqrt(6/shape_sum),np.sqrt(6/shape_sum), [Mi, Mo])

In [0]:
embedding_size_w = 100
vocab_size = len(word2idx)
n_neg_samples = 20
learning_rate = 10e-5
epochs = 1001
batch_size=10000
mu = 0.99
window_size = 5

In [0]:
# Define placeholders for training 
train_X = tf.placeholder(tf.int32, shape=[batch_size, 1])
train_label = tf.placeholder(tf.int32, shape=[batch_size, None])

In [0]:
# Define matrix for doc_embedding and word_embedding 
W1 = tf.Variable(init_weight(vocab_size, embedding_size_w), name="W1", dtype=tf.float32)

In [0]:
# Define weights for the output unit 
W2 = tf.Variable(init_weight(vocab_size, embedding_size_w), name="W2", dtype=tf.float32)
biases = tf.Variable(tf.zeros(vocab_size))

In [21]:
print(train_X.get_shape(), train_label.get_shape(), W1.get_shape(), W2.get_shape())

(10000, 1) (10000, ?) (1001, 100) (1001, 100)


In [0]:
embed = tf.nn.embedding_lookup(W1, train_X[0])

In [0]:
loss = tf.nn.sampled_softmax_loss(weights=W2, \
                                  biases=biases, \
                                  labels=train_label, \
                                  inputs=embed, \
                                  num_sampled=n_neg_samples, \
                                  num_classes=vocab_size, 
                                  num_true=window_size*2)

In [0]:
loss = tf.reduce_mean(loss)

In [0]:
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=mu).minimize(loss)
#optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)


global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 10e-5
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.96, staircase=True)
# Passing global_step to minimize() will increment it at each step.
optimizer = (
    tf.train.MomentumOptimizer(learning_rate, momentum=mu).minimize(loss, global_step=global_step)
)


In [0]:
saver = tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    average_loss = 0
    
    for step in range(epochs):
        epoch_error = 0.0
        temp_X , temp_labels = getNextBatchSkipGram(window_size=5, bi_grams_=training_data)
        
        feed_dict = {train_X : temp_X, train_label : temp_labels}
        
        op, l = sess.run([optimizer, loss], 
                                    feed_dict=feed_dict)
        
        epoch_error += l
                
        if step % 100 == 0:
            print ("Error at epoch : ", step, " = ", epoch_error)
            
    save_path = saver.save(sess, "./models/model_skipgram_model.ckpt")
    print("Model saved in file: %s" % save_path)

Error at epoch :  0  =  4.074723243713379
Error at epoch :  100  =  4.227557182312012
Error at epoch :  200  =  3.8662407398223877


##### Embeddings 

In [0]:
W1_embedding = None
W2_embedding = None 

with tf.Session() as sess:
    saver = tf.train.Saver()
    # Restore variables from disk.
    saver.restore(sess, "./models/model_skipgram_model.ckpt")
    print("Model restored.")
    
    # Normalize word2vec 
    W1_embedding = W1.eval()
    
    # Normalize word2vec 
    W2_embedding = W2.eval()

In [0]:
word2vec = np.mean([W1_embedding, W2_embedding], axis=0)

##### Projection of embeddings using t-SNE 

In [0]:
idx2word = {v:k for k, v in word2idx.items()}

In [0]:
from sklearn.manifold import TSNE
model = TSNE()
Z = model.fit_transform(word2vec) 

In [0]:
#plt.scatter(Z[:,0], Z[:,1])
for i in range(len(idx2word)):
    try:
        plt.annotate(s=idx2word[i].encode("utf8"), xy=(Z[i,0], Z[i,1]))
    except:
        print ("bad string:", idx2word[i])
plt.show()